In [1]:
import glob
import pandas as pd

rawData = []
outfile = "results.pkl"

In [3]:
for file in glob.glob("*.log"):
    print(file)
    workloadParts = file.split("_")
    platform = workloadParts[0]
    regression = int(workloadParts[1])
    wrapperCount = int(workloadParts[2])
    callsPerFunction = workloadParts[3]
    iterations = int(workloadParts[4].split(".")[0])
    #    with open(file) as logfile:
    with open(file) as logfile:
        for line in logfile:
            if line.__contains__("faaster"):
                line = line[:-4]
                if line.__contains__("modeA") and not line.__contains__("cold start"):
                    
                    line = line.strip()
                    parts2 = line.split(" ")
                    #print(line)
                    row = {}
                    row["platform"] = platform
                    row["regression"] = regression
                    row["wrapperCount"] = wrapperCount
                    row["callsPerFunction"] = callsPerFunction
                    row["iterations"] = iterations

                    row["mode"] = parts2[2]

                    f1Durations = []
                    f2Durations = []
                    f2Found = False

                    for i in range(4,len(parts2)):
                        if parts2[i] == "f2":
                            f2Found = True
                            continue
                        if f2Found:
                            if len(f2Durations) < len(f1Durations):
                                f2Durations.append(int(parts2[i]))
                        else:
                            f1Durations.append(int(parts2[i]))

                    for i in range(0, len(f1Durations)):
                        #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                        row["f1"] = f1Durations[i]
                        row["f2"] = f2Durations[i]
                        row["change"] = f2Durations[i] / f1Durations[i]
                        rawData.append(row)
                if line.__contains__("modeB") and not line.__contains__("cold start"):
                    line = line.strip()
                    parts2 = line.split(" ")
                    #print(line)
                    row = {}
                    row["platform"] = platform
                    row["regression"] = regression
                    row["wrapperCount"] = wrapperCount
                    row["callsPerFunction"] = callsPerFunction
                    row["iterations"] = iterations
                    row["mode"] = parts2[2]

                    f1Durations = []

                    for i in range(4,len(parts2)):
                        if parts2[i] == "f2":
                            break
                        else:
                            f1Durations.append(int(parts2[i]))

                    for i in range(0, len(f1Durations)):
                        #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                        row["f1"] = f1Durations[i]
                        row["f2"] = 0
                        row["change"] = 0
                        rawData.append(row)

                if line.__contains__("modeC") and not line.__contains__("cold start"):
                    line = line.strip()
                    parts2 = line.split(" ")
                    #print(line)
                    row = {}
                    row["platform"] = platform
                    row["regression"] = regression
                    row["wrapperCount"] = wrapperCount
                    row["callsPerFunction"] = callsPerFunction
                    row["iterations"] = iterations
                    row["mode"] = parts2[2]

                    f2Durations = []

                    for i in range(5,len(parts2)):
                        if parts2[i] == "\",":
                            break
                        else:
                            f2Durations.append(int(parts2[i]))

                    for i in range(0, len(f2Durations)):
                        #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                        row["f1"] = 0
                        row["f2"] = f2Durations[i]
                        row["change"] = 0
                        rawData.append(row)




aws_0_10_5_3.log
aws_0_10_10_3.log
aws_0_5_25_3.log
aws_0_5_10_3.log
aws_0_10_25_3.log
aws_0_5_5_3.log


In [4]:
df_results = pd.DataFrame(rawData)
df_results.head()
df_results.describe()

df_results.to_pickle(outfile)
#df_results.to_csv("nils2.csv")